# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
weather_df = pd.read_csv('../WeatherPy/output/cities.csv')
weather_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
humid_max = weather_df['Humidity (%)'].max()

heat_layer = gmaps.heatmap_layer(weather_df[['Latitude', 'Longitude']], weights= weather_df['Humidity (%)'], 
                                 dissipating=False, max_intensity=humid_max,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
best_df = weather_df.loc[(weather_df['Temperature \N{DEGREE SIGN}F'] > 0) & 
                          (weather_df['Temperature \N{DEGREE SIGN}F'] < 30) &
                          (weather_df['Humidity (%)'] < 90) &
                          (weather_df['Cloud Coverage (%)'] == 0)
                         ]
print(best_df.shape)
best_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
best_df['Hotel Name'] = ""
best_df['Country'] = ""
best_df = best_df.reset_index()
best_df.head()

In [ ]:
# geo coordinates
target_radius = 10000 * 1000
target_type = "lodging"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = []
countries = []

for index, row in best_df.iterrows():
    
    target_coords = f'{row["Latitude"]},{row["Longitude"]}'

    # set up a parameters dictionary
    params = {
    "location": target_coords,    
    "type": target_type,   
    "radius": target_radius,
    "key": g_key   
    }
    
    print(target_coords)
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()

    try:
        hotels_name = response['results'][0]['name']
        country_response = response['results'][0]['plus_code']['compound_code']
        countries_name = country_response.split(", ",1)[1]
        
        hotels.append(hotel_name)
        countries.append(country_name)
        
    except:
        
        print(f'No results for #{index}: {row["City"]}. Skipping')
        hotels.append("")
        countries.append("")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotels_info = [info_box_template.format(**row) for index, row in best_df.iterrows()]
locations = best_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='red',
    stroke_color='red', 
    info_box_content=hotels_info
)


fig.add_layer(hotel_layer)

fig